<center>
    <h1>
            Probabilidad y Estadística
    </h1>
    <h2>
            Trabajo Práctico
    </h2>
</center>

<table>
<thead>
  <tr>
    <th>Integrante</th>
    <th>Legajo</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>Cintia Capece</td>
    <td>xxxxx</td>
  </tr>
  <tr>
    <td>Nicolás Bellesi</td>
    <td>xxxxx</td>
  </tr>
  <tr>
    <td>Lucas Nevado</td>
    <td>40768</td>
  </tr>
</tbody>
</table>

## Índice

* [Setup](#setup)
* [Parte 1: Simulación](#parte_1)
    * [1.1 Generador Bernoulli](#p_1_1)
    * [1.2 Generador Binomial](#p_1_2)
    * [1.3 Generador Exponencial](#p_1_3)
    * [1.4 Generador Normal (y Normal Estándar)](#p_1_4)
* [Parte 2: Estadística descriptiva](#parte_2)
    * [2.3 Muestra n=50 Bin(10, 0.3) y función de distribución empírica](#p_2_3)
    * [2.4 Nueva muestra a partir de la función de distribución empírica. Media y varianza muestral](#p_2_4)
    * [2.5 Repetición del experimento](#p_2_5)
* [Parte 4: Estadística inferencial](#parte_4)
* [Bibliografía](#bibliografia)

---

## Setup<a class="anchor" id="setup"></a>

In [1]:
import random
import math
import numpy as np
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import row
from bokeh.models import Legend, LegendItem


output_notebook()

def muestra_gen(muestra_tam, distribucion_gen, *distribucion_args):
    """
    muestra_tam: Tamaño de la muestra
    distribucion_gen: Generador de una distribución, ej: exponencial_gen
    distribucion_args: Argumentos necesarios para la distribución, ej: 0.5
    """
    muestra = []
    for i in range(muestra_tam):
        valor = distribucion_gen(*distribucion_args)
        muestra.append(valor)

    return muestra

def crear_histograma(muestra, ancho_de_banda = 0):
    """
    A partir de una muestra, devuelve una histograma de bokeh
    El ancho de banda es opcional
    """
    if ancho_de_banda == 0:
        hist, edges = np.histogram(muestra, density=True)
    else:
        hist, edges = np.histogram(muestra, density=True, bins=math.floor((max(muestra) - min(muestra)) / ancho_de_banda))

    p = figure()
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
    return p

def agregar_media(histograma, media):
    histograma.ray(x=[media], y=[0], length=0, angle=(np.pi) / 2, line_width=3, line_color="red", legend_label="x̄ = " + str(media))
    return histograma
    

Loading BokehJS ...

## Parte 1: Simulación<a class="anchor" id="parte_1"></a>

### 1) Generador Bernoulli<a class="anchor" id="p_1_1"></a>

In [2]:
def bernoulli_gen(p):
    uniforme = random.uniform(0, 1)
    if uniforme < p:
        return 1
    else:
        return 0

In [3]:
bernoulli_gen(0.75)

1

### 2) Generador Binomial<a class="anchor" id="p_1_2"></a>

In [4]:
def binomial_gen(n, p):
    exitos = 0
    for i in range(n):
        exitos += bernoulli_gen(p)

    return exitos

In [5]:
binomial_gen(10, 0.5)

7

### 3) Generador Exponencial<a class="anchor" id="p_1_3"></a>

In [6]:
def exponencial_gen(λ):
    """
    Utilizamos el método de la función inversa
    """
    uniforme = random.uniform(0, 1)
    
    exponencial = - (1 / λ) * math.log(uniforme)

    return exponencial

In [7]:
exponencial_gen(3)

0.9048601736211248

### 4) Generador Normal (y Normal Estándar)<a class="anchor" id="p_1_4"></a>

In [8]:
def std_normal_gen():
    """
    Utilizamos el método de Box-Muller
    """
    uniforme_U = random.uniform(0, 1)
    uniforme_V = random.uniform(0, 1)

    std_normal = math.sqrt(-2 * math.log(uniforme_U)) * math.cos(2 * math.pi * uniforme_V)
    
    return std_normal

In [9]:
std_normal_gen()

0.8214923167777632

In [10]:
def normal_gen(μ = 0, σ_2 = 1):
    σ = math.sqrt(σ_2)
    normal = std_normal_gen() * σ + μ

    return normal

In [11]:
normal_gen(1, 1)

1.0691950668708032

## Parte 2: Estadística descriptiva<a class="anchor" id="parte_2"></a>

### 3) Muestra n=50 Bin(10, 0.3) y función de distribución empírica<a class="anchor" id="p_2_3"></a>

In [12]:
muestra_bin = muestra_gen(50, binomial_gen, 10, 0.3)

def fde(muestra, x):
    muestra_ordenada = np.sort(muestra)
    N = len(muestra)
    return np.searchsorted(muestra_ordenada, x) / N

fde(muestra_bin, 3)

0.42

### 4) Nueva muestra a partir de la función de distribución empírica. Media y varianza muestral<a class="anchor" id="p_2_4"></a>

In [13]:
def muestra_fde(muestra):
    muestra_fde = []
    for i in muestra_bin:
        v = fde(muestra_bin, i)
        muestra_fde.append(v)
    return muestra_fde

muestra_fde_bin = muestra_fde(muestra_bin)
    
def media_muestral(muestra):
    return sum(muestra) / len(muestra)

def varianza_muestral(muestra):
    media = media_muestral(muestra)
    return sum([(x - media) ** 2 for x in muestra]) / (len(muestra) - 1)

media_muestral_bin = round(media_muestral(muestra_bin), 3)
media_muestral_fde_bin = round(media_muestral(muestra_fde_bin), 3)
varianza_muestral_bin = round(varianza_muestral(muestra_bin), 3)
varianza_muestral_fde_bin = round(varianza_muestral(muestra_fde_bin), 3)

print("Media Muestral Bin(10, 0.3): ", media_muestral_bin)
print("Media Muestral FDE(Bin(10, 0.3)): ", media_muestral_fde_bin)

print("Varianza Muestral Bin(10, 0.3): ", varianza_muestral_bin)
print("Varianza Muestral FDE(Bin(10, 0.3)): ", varianza_muestral_fde_bin)

hist1 = crear_histograma(muestra_bin)
agregar_media(hist1, media_muestral_bin)

hist2 = crear_histograma(muestra_fde_bin)
agregar_media(hist2, media_muestral_fde_bin)

show(row(hist1, hist2, sizing_mode='scale_both'))

Media Muestral Bin(10, 0.3):  2.98
Media Muestral FDE(Bin(10, 0.3)):  0.405
Varianza Muestral Bin(10, 0.3):  2.265
Varianza Muestral FDE(Bin(10, 0.3)):  0.09


### 5) Repetición del experimento<a class="anchor" id="p_2_5"></a>

In [14]:
# Primera repetición
muestra_bin = muestra_gen(50, binomial_gen, 10, 0.3)
muestra_fde_bin = muestra_fde(muestra_bin)


media_muestral_bin = round(media_muestral(muestra_bin), 3)
media_muestral_fde_bin = round(media_muestral(muestra_fde_bin), 3)
varianza_muestral_bin = round(varianza_muestral(muestra_bin), 3)
varianza_muestral_fde_bin = round(varianza_muestral(muestra_fde_bin), 3)

print("2 - Media Muestral Bin(10, 0.3): ", media_muestral_bin)
print("2 - Media Muestral FDE(Bin(10, 0.3)): ", media_muestral_fde_bin)

print("2 - Varianza Muestral Bin(10, 0.3): ", varianza_muestral_bin)
print("2 - Varianza Muestral FDE(Bin(10, 0.3)): ", varianza_muestral_fde_bin)

hist1 = crear_histograma(muestra_bin)
agregar_media(hist1, media_muestral_bin)

hist2 = crear_histograma(muestra_fde_bin)
agregar_media(hist2, media_muestral_fde_bin)

show(row(hist1, hist2, sizing_mode='scale_both'))

# Segunda repetición
muestra_bin = muestra_gen(50, binomial_gen, 10, 0.3)
muestra_fde_bin = muestra_fde(muestra_bin)

media_muestral_bin = round(media_muestral(muestra_bin), 3)
media_muestral_fde_bin = round(media_muestral(muestra_fde_bin), 3)
varianza_muestral_bin = round(varianza_muestral(muestra_bin), 3)
varianza_muestral_fde_bin = round(varianza_muestral(muestra_fde_bin), 3)

print("3 - Media Muestral Bin(10, 0.3): ", media_muestral_bin)
print("3 - Media Muestral FDE(Bin(10, 0.3)): ", media_muestral_fde_bin)

print("3 - Varianza Muestral Bin(10, 0.3): ", varianza_muestral_bin)
print("3 - Varianza Muestral FDE(Bin(10, 0.3)): ", varianza_muestral_fde_bin)

hist1 = crear_histograma(muestra_bin)
agregar_media(hist1, media_muestral_bin)

hist2 = crear_histograma(muestra_fde_bin)
agregar_media(hist2, media_muestral_fde_bin)

show(row(hist1, hist2, sizing_mode='scale_both'))

2 - Media Muestral Bin(10, 0.3):  2.7
2 - Media Muestral FDE(Bin(10, 0.3)):  0.399
2 - Varianza Muestral Bin(10, 0.3):  2.01
2 - Varianza Muestral FDE(Bin(10, 0.3)):  0.081


3 - Media Muestral Bin(10, 0.3):  3.22
3 - Media Muestral FDE(Bin(10, 0.3)):  0.375
3 - Varianza Muestral Bin(10, 0.3):  1.889
3 - Varianza Muestral FDE(Bin(10, 0.3)):  0.099


## Parte 4: Estadística inferencial<a class="anchor" id="parte_4"></a>

In [15]:
muestra_N_100_5_10 = muestra_gen(10, normal_gen, 100, 5)
muestra_N_100_5_30 = muestra_gen(30, normal_gen, 100, 5)
print("Media Muestral N(100, 5), n=10: ", round(media_muestral(muestra_N_100_5_10), 3))
print("Media Muestral N(100, 5), n=30: ", round(media_muestral(muestra_N_100_5_30), 3))
print("Varianza Muestral N(100, 5), n=10: ", round(varianza_muestral(muestra_N_100_5_10), 3))
print("Varianza Muestral N(100, 5), n=30: ", round(varianza_muestral(muestra_N_100_5_30), 3))

def intervalo_de_confianza_mu_varianza_conocida(nivel_porcentaje, muestra, varianza):
    if nivel_porcentaje == 95:
        z = 1.96
    if nivel_porcentaje == 98:
        z = 2.32
    n = len(muestra)
    σ = math.sqrt(varianza)
    limite_inferior = round(media_muestral(muestra) - z * (σ / math.sqrt(n)), 3)
    limite_superior = round(media_muestral(muestra) + z * (σ / math.sqrt(n)), 3)
    return [limite_inferior, limite_superior]

print(intervalo_de_confianza_mu_varianza_conocida(95, muestra_N_100_5_30, 5))
print(intervalo_de_confianza_mu_varianza_conocida(98, muestra_N_100_5_30, 5))

def intervalo_de_confianza_mu_varianza_desconocida(nivel_porcentaje, muestra, S_2):
    n = len(muestra)
    if nivel_porcentaje == 95:
        if n == 10:
            t = 2.262
        if n == 30:
            t = 2.045
    if nivel_porcentaje == 98:
        if n == 10:
            t = 2.821
        if n == 30:
            t = 2.462
    S = math.sqrt(S_2)
    limite_inferior = round(media_muestral(muestra) - t * (S / math.sqrt(n)), 3)
    limite_superior = round(media_muestral(muestra) + t * (S / math.sqrt(n)), 3)
    return [limite_inferior, limite_superior]

print(intervalo_de_confianza_mu_varianza_desconocida(95, muestra_N_100_5_30, 5))
print(intervalo_de_confianza_mu_varianza_desconocida(98, muestra_N_100_5_30, 5))


Media Muestral N(100, 5), n=10:  99.245
Media Muestral N(100, 5), n=30:  99.728
Varianza Muestral N(100, 5), n=10:  2.118
Varianza Muestral N(100, 5), n=30:  5.539
[98.927, 100.528]
[98.781, 100.675]
[98.893, 100.563]
[98.723, 100.733]


## Prueba histograma

In [16]:
muestra_normal_estandar_1000 = muestra_gen(1000, std_normal_gen)
histograma = crear_histograma(muestra_normal_estandar_1000)
show(histograma)

# También se puede mostrar en una fila varios histogramas
histograma_con_ancho = crear_histograma(muestra_normal_estandar_1000, 0.5)
show(row(histograma, histograma_con_ancho, sizing_mode='scale_both'))

## Bibliografía<a class="anchor" id="bibliografia"></a>